<a href="https://colab.research.google.com/github/francji1/01RAD/blob/main/R/01RAD_Ex05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD Exercise 05


Today's exercise
 * Numerical Linear Algebra view-
 * Residuals

In [ ]:
# get requirements for this exercise
list_of_packages <- c("tidyverse", "lattice", "pwr", "MASS",
                      "GGally","scatterplot3d","gridExtra",
                      "nortest","lmtest") # "car","colorspace","RColorBrewer","pracma"
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)


In [ ]:
######################
# check our settings
# getwd()
# print(.libPaths())
# print(sessionInfo())
print(version)

tibble::tibble(
  Package = names(installed.packages()[,3]),
  Version = unname(installed.packages()[,3])
)


## Numerical algebra view

In [ ]:
# Let's take the same model from the previous lectures
head(cars)
cars_lm2 <- lm(dist ~  speed , data = cars)
summary(cars_lm2)


Compute residuals of the model: $r = Y - \hat{Y} = Y - X \hat{\beta}$

In [ ]:
# computed residuals
r <- residuals(cars_lm2) 
# alternatives
# r <- resid(cars_lm2),
# r <- cars_lm2$residuals
# r <- cars$dist - fitted(cars_lm2)  
#  cars_lm2$fitted.values  # Fitted values of response varaible (here cars$dist)

hist(r)
qqnorm(r)
qqline(r)



In [ ]:
mean(cars_lm2$residuals)
sqrt(var(cars_lm2$residuals))
sd(cars_lm2$residuals)
deviance(cars_lm2)/cars_lm2$df.residua
summary(cars_lm2)



In [ ]:
#coefficients(cars_lm2)    # Regression coefficens
# cars_lm2$df.residual    # Residuals degrees of freedom 

In [ ]:
X <- cbind(rep(1,times=length(cars$speed)), cars$speed)
Y <- cars$dist
(XtX  <- t(X) %*% X) 
(XtY  <- t(X) %*% Y)
b_ols <- solve(XtX, XtY)  
cbind(b_ols,coefficients(cars_lm2))


In [ ]:
## QR decomposition   
qr(X)             # object of class qr
R = qr.R(qr(X))
Qt = t((qr.Q(qr(X))))
Q = (qr.Q(qr(X)))
#(Q%*%R == X)


In [ ]:
qr.solve(X,Y)    
R%*%qr.solve(X,Y) ==  Qt%*%Y
round(qr.R(qr(X))%*%qr.solve(X,Y),5) ==  round(t((qr.Q(qr(X))))%*%Y,5)


In [ ]:
# Q*Q^T = I => Q^T = Q^-1
b_ols = solve(R)%*%Qt%*%Y
b_ols

In [ ]:
## as MATLAB / bslash function ;)
bslash <- function(X, Y) {
    qr.coef(qr(X), Y)
}
b_ols2 <- bslash(X, Y)
b_ols2


In [ ]:
## QR decomposition from lm object:
#  Q matrix  is orthogonal, i.e. columns are orthogonal vectors
Q <- qr.Q(cars_lm2$qr)  
H <- qr.Q(cars_lm2$qr) %*% t(qr.Q(cars_lm2$qr)) # H = QtQ
dim(H)



In [ ]:
# Hat matrix H = projection matrix
# orthogonal projection onto the column space of the matrix X 
H <- X %*% solve(XtX) %*% t(X)    
dim(H)
H[1:5,1:5]



In [ ]:
#############
# TASK
# How looks H eigenvalues? 
round(eigen(H)$values,10)
# What is H%*%H ? How do we call matrix with this property?
sum(round(H%*%H - H,5))
round(t(H) - H,10)


In [ ]:
dim(H)
dim(X)
############
## prediction:  hat_Y = H%*%Y

In [ ]:
#Question + Task
## Complement of the projection matrix H
M <- diag(nrow(H)) - H            
# Why is matrix M called  residual maker matrix ?
# Compute residuals by the help of M


In [ ]:
cars_lm2$fitted.values  # Fitted values of response variable (here cars$dist)

In [ ]:

# TASK
# compute fitted values manually by the help of original data set and cars_lm2$coef
# compare results with output from the function lm_object$fitted.values or fitted(lm_object)
# compare with result obtained by H matrix
fittedY1  = cars_lm2$fitted.values
fittedY1b = fitted(cars_lm2)
fittedY2  = X%*%coef(cars_lm2)
fittedY3  = H%*%Y



In [ ]:
#The eigenvalues of H  consist of r ones and n-r zeros.
#The eigenvalues of M consist  of n-r ones and r  zeros.
round(eigen(H)$values,10)
round(eigen(M)$values,10)



In [ ]:
# H and M are idempotent: 
sum(round(H%*%H,5) ==round(H,5))
sum(round(M%*%M,5) ==round(M,5))

cbind(fittedY1, fittedY1b, fittedY2,fittedY3)[1:10,]  

In [ ]:
# Similar for residuals
residuals(cars_lm2)
resid_2a = cars[,"dist"] - fittedY3   # manually from the definition
resid_2b = residuals(cars_lm2)          # by R function
resid_2c = M%*%Y
cbind(resid_2a, resid_2b, resid_2c)[1:10,]  


## Residual  Analysis

In [ ]:
# Let's generate some data
set.seed(21)
n    <- 60
X0   <- rep(1,n)
X1   <- runif(n,10,40)
X    <- cbind(X0,X1)
e    <- rnorm(n,0,2)
beta <- c(4,2)
Y    <- X%*%matrix(beta) + e
df_m1 <- data.frame(Y = Y, X = X1)


In [ ]:

# Summary and visualisation of our dataset
ggpairs(df_m1)
summary(df_m1)


In [ ]:

# OLS estimation of regression coefficients
m1 <- lm(Y ~ X, df_m1)
summary(m1)


In [ ]:

# Fast post-hoc analysis of residuals
opar <- par(mfrow=c(2,2))
plot(m1)
par(opar)



In [ ]:

hist(resid(m1))
fitted(m1)


In [ ]:

#residuals vs. fitted
plot(residuals(m1) ~ fitted(m1))



In [ ]:

#########################################
# How R computes different residuals? 
# R provides built-in functions rstandard() and rstudent() as as part of influence.measures 
# They re-normalize the residuals to have unit variance,
# using an overall and leave-one-out measure of the error variance respectively.


In [ ]:

# manual computation of standartized residuals : help(rstandard)
# R standartized residual is ordinary residual divided by an estimate of its standard deviation:
r_standard = residuals(m1)/(sigma(m1)*sqrt(1-hatvalues(m1)))
# comparison with built-in function
r_standard - rstandard(m1)


#Studentized residuals: "externally studentized residuals" or "deleted residuals," 
#Studentized residuals for any given data point are computed from a model fit 
# to every other data point except the one in question. 

# compare with slide 1, lecture 5
anova(m1)

A = sqrt(1 - (1/n + (X1-mean(X1))^2/sum((X1-mean(X1))^2)))
r_stud <- residuals(m1) / (sigma(m1)*A)

round(rstudent(m1) - r_stud,4)
round(rstandard(m1) - r_stud,4)

rstandard(m1, type = "predictive")
# we will come back to standardized and studentized residuals later 
# in outlier diagnostic session




In [ ]:

# Plot with ggplot, add residuals to dataset

df_m1 <- df_m1  %>% 
  mutate(predicted = predict(m1),
         residuals = residuals(m1),
         rstandard = rstandard(m1))

p1<-ggplot(data=df_m1,mapping=aes(x=predicted,y=residuals)) +
  geom_point() +
  geom_hline(yintercept=0,linetype="dashed")

p2<-ggplot(data=df_m1,mapping=aes(x=Y,y=residuals)) +
  geom_point() +
  geom_hline(yintercept=0,linetype="dashed")

p3<-ggplot(data=df_m1,mapping=aes(x=predicted,y=rstandard)) +
  geom_point() +
  geom_hline(yintercept=0,linetype="dashed")

p4<-ggplot(data=df_m1,mapping=aes(x=Y,y=rstandard)) +
  geom_point() +
  geom_hline(yintercept=0,linetype="dashed")

ggarrange(p1, p2, p3, p4, labels = c("A", "B","C","D"), common.legend = TRUE, legend = "bottom")



In [ ]:

# Visualization of residuals (orthogonal distance to be minimized by OLS method)

ggplot(df_m1, aes(x = X, y = Y)) +
  geom_smooth(method = "lm", se = FALSE, color = "lightgrey") +
  geom_segment(aes(xend = X, yend = predicted), alpha = .2) +
  geom_point(aes(color = residuals)) +  
  scale_color_gradient2(low = "blue", mid = "white", high = "red") +  # Colors to use here
  # geom_point() + 
  # guides(color = FALSE) +
  geom_point(aes(y = predicted), shape = 1) +
  theme_bw()



Lets play with regression model and obtain different shapes of residuals

In [ ]:

X1   <- runif(n,0,1)
X    <-cbind(X0,sin(2*pi*X1))
Y    <- X%*%matrix(c(1,1)) + rnorm(n,0,(0.1)^2)
df_m1 <- data.frame(Y = Y, X = X1)
summary(df_m1)
ggpairs(df_m1)
m1 <- lm(Y ~ X, df_m1)
opar <- par(mfrow=c(2,2))
plot(m1)
par(opar)



In [ ]:

X1   <- runif(n,0,10)
X    <- cbind(X0,log(0.4 + X1))
Y    <- X%*%matrix(c(1,1)) + rnorm(n,0,0.1)
df_m1<- data.frame(Y = Y, X = X1)
ggpairs(df_m1)
m1 <- lm(Y ~ X, df_m1)
opar <- par(mfrow=c(2,2))
plot(m1)
par(opar)


In [ ]:

X0   <- rep(1,n)
X1   <- runif(n,10,40)
X    <- cbind(X0,X1)
Y   <- X%*%matrix(beta) + (rnorm(n,0,(X1/10)^2)+X1/10)
df_m1 <- data.frame(Y = Y, X = X1)
ggpairs(df_m1)
summary(df_m1)
m1 <- lm(Y ~ X, df_m1)
opar <- par(mfrow=c(2,2))
plot(m1)
par(opar)
# repeat plots ;)